In [11]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

# 1. 환경 변수 로딩
load_dotenv()

# 2. Pinecone 연결
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "investment-analysis-final"

# 3. 인덱스가 없으면 생성
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI 임베딩 차원 수
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"  # 👉 실제 Pinecone 인덱스 지역
        )
    )

# 4. 인덱스 연결
index = pc.Index(index_name)

# 5. OpenAI 임베딩 모델 준비
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# 6. 문서 분할 전략 정의
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=80,
    separators=["\n\n", "\n", " ", ""]
)

# 7. PDF 파일 리스트
pdf_files = [
    ("AI News.pdf", "ai_news"),
    ("Startup Evaluation Metrics.pdf", "startup_metrics"),
    ("VoyagerX Introduction.pdf", "voyagerx_intro"),
    ("Video Stt Script.pdf", "video_stt_script")  # ✅ 추가된 PDF
]

all_chunks = []

# 8. 각 PDF 로딩, 분할, 메타데이터 부여
for filename, doc_id in pdf_files:
    loader = PyPDFLoader(filename)
    pages = loader.load_and_split()

    for i, doc in enumerate(pages):
        doc.metadata["source"] = doc_id
        doc.metadata["page"] = i + 1

    chunks = text_splitter.split_documents(pages)
    all_chunks.extend(chunks)

# 9. Pinecone에 업로드
vectorstore = PineconeVectorStore.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    index_name=index_name
)


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 365 0 (offset 0)


In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "남세동 CEO에 대해 알려줘."
)

response = llm.invoke(prompt.format())
print("🧠 GPT 단독 응답:\n", response.content)

🧠 GPT 단독 응답:
 남세동 CEO는 한국의 기업인으로, 주식회사 네이버의 현 CEO이며 이사회 의장이기도 합니다. 그는 네이버의 창업 멤버 중 한 명으로, 회사의 성장과 발전에 큰 역할을 해왔습니다. 또한 인터넷 기업의 선구자로서, 한국의 IT 산업을 선도하는 인물로 평가받고 있습니다. 현재 네이버를 한국을 넘어 글로벌 기업으로 성장시키기 위해 노력하고 있습니다.


In [15]:
query = "남세동 CEO에 대해서 알려줘"

# 유사 문서 검색
retrieved_docs = vectorstore.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# 프롬프트에 문서 포함
final_prompt = f"""
다음 문서를 참고해서 질문에 답해주세요.

문서:
{context}

질문: {query}
"""

response = llm.invoke(final_prompt)
print("📚 벡터 DB + GPT 응답:\n", response.content)


📚 벡터 DB + GPT 응답:
 남세동은 보이저엑스의 CEO이며, 자기주도적인 직원을 채용하기 위해 4단계의 채용 과정을 거치며, 지원자의 과거 경험과 정직성을 중요하게 평가합니다. 또한, 언러닝(Unlearning)의 중요성을 강조하며, 회사와 개인이 동반의 개념이라는 사실을 강조하고 있습니다. 회사의 성장과 개인의 성장 사이의 조화와 밸런스를 맞추는 것을 중요시하고 있으며, 회사에 헌신하지 말라고 이야기하며, 맹목적으로 지시에 따르는 것이 아닌 자기주도적으로 일하고 성장할 수 있는 문화를 만들어가고 있습니다.
